## 七月在线电商点击率预估特训营结业考试
#### 考试说明:
- 起止时间：<font color=red><b>2020年4月9日10:00 - 2020年4月16日22:00</b></font>，<b>逾期不接受补考</b>
- 考试提交方式：<font color=red><b> 将试卷下载到本地作答</b></font>，本地作答完成后将文件命名为<font color=red><b>"姓名_学号"</b></font>并上传到：http://47.93.208.249:9925/tree/0.Teacher/Exam/Stage1 
- 注意事项：为确保同学们真正了解自身对这次课程的掌握程度，<font color=red><b>请勿翻阅抄袭，移动，更改</b></font>其它同学的试卷，如发现按0分处理。
- 请同学在下方同学姓名处填写自己的姓名，批改人和最终得分处不用填写，试卷总分为100分。

## 一、理论题（40分）

#### 1、为什么LR可以用来作CTR预估？（10分）

1. CTR预估问题属于二分类任务，LR是二分类模型
2. 点击率预估很多模型可以做，但是LR实现简单，方便快速迭代和并行，适合工业界使用。

#### 2、 FM中二阶交叉项的权值矩阵大小为？（其中N为特征数量，K为隐因子数量）（10分）

A N x N

B N x K

C K x K

答案：B

#### 3、请你说一下FNN的优点和缺点，各列举一条（10分）

优点：引入DNN对特征进行更高阶组合，减少特征工程，能在一定程度上增强FM的学习能力，这种尝试为后续深度推荐模型的发展提供了新的思路。

缺点：两阶段训练模式，在应用过程中不方便，且模型能力受限于FM表征能力的上限。FNN专注于高阶组合特征，但是却没有将低阶特征纳入模型。

#### 4、请说一下 PNN 中 Product Layer 非线性部分的两种形式以及他们的区别。（10分）

PNN的两种形式分别是：内积运算的IPNN和外积运算的OPNN
两种形式的区别就是特征交叉的运算方式不同，IPNN采用的是内积运算，OPNN采用的是外积运算。

## 二、代码题（60分）

In [6]:
import numpy as np
import tensorflow as tf
import utils
from main import *
dtype = utils.DTYPE

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
[[51, 0, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44], [87, 0, 27, 28, 45, 46, 47, 32, 48, 49, 35, 36, 50, 38, 39, 40, 51, 42, 52, 53, 44], [33, 0, 27, 28, 54, 55, 56, 32, 57, 58, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44], [65, 0, 27, 28, 59, 60, 61, 32, 62, 63, 35, 36, 37, 38, 39, 40, 41, 42, 64, 43, 44], [238, 0, 27, 28, 65, 66, 67, 32, 68, 69, 70, 71, 50, 38, 72, 40, 51, 42, 73, 74, 75, 76, 43, 77, 78, 79, 52, 80, 81, 82, 83, 84, 85], [65, 0, 27, 28, 86, 87, 88, 32, 62, 63, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44], [6, 0, 27, 28, 89, 90, 91, 32, 92, 93, 35, 36, 37, 38, 39, 40, 51, 42, 43, 44], [129, 0, 27, 28, 94, 95, 96, 32, 97, 98, 35, 36, 37, 38, 39, 40, 41, 42, 74, 77, 78, 64, 84, 43, 76, 99, 100, 75, 82, 44], [57, 0, 27, 28, 101, 87, 88, 32, 102, 103, 70, 71, 37, 38, 72, 40, 41, 42, 77], [81, 0, 27, 28, 104, 55, 105, 32, 106, 107, 108, 109, 37, 38, 110, 40, 51, 42, 111, 85], [55, 0, 27, 28, 112, 113, 114, 32, 115,

<font color=red>注意：请下载试卷后在自己的电脑上运行代码，另外当前目录下的 main.py、utils.py 和 data 文件夹（即第二阶段的 data 文件夹）需要一同下载</font>

In [7]:
class Model:
    def __init__(self):
        self.sess = None
        self.X = None
        self.y = None
        self.layer_keeps = None
        self.vars = None
        self.keep_prob_train = None
        self.keep_prob_test = None

    def run(self, fetches, X=None, y=None, mode='train'):
            feed_dict = {}
            if type(self.X) is list:
                for i in range(len(X)):
                    feed_dict[self.X[i]] = X[i]
            else:
                feed_dict[self.X] = X
            if y is not None:
                feed_dict[self.y] = y
            if self.layer_keeps is not None:
                if mode == 'train':
                    feed_dict[self.layer_keeps] = self.keep_prob_train
                elif mode == 'test':
                    feed_dict[self.layer_keeps] = self.keep_prob_test
            return self.sess.run(fetches, feed_dict)

    def dump(self, model_path):
        var_map = {}
        for name, var in self.vars.iteritems():
            var_map[name] = self.run(var)
        pkl.dump(var_map, open(model_path, 'wb'))
        print('model dumped at', model_path)

#### 1. 请你补全下面 LR 类中缺失的代码，然后运行程序，输出AUC结果（30分）

In [8]:
class LR(Model):
    def __init__(self, input_dim=None, output_dim=1, init_path=None, opt_algo='gd', learning_rate=1e-2, l2_weight=0,
                 random_seed=None):
        Model.__init__(self)
        init_vars = [('w', [input_dim, output_dim], 'tnormal', dtype),
                     ('b', [output_dim], 'zero', dtype)]
        self.graph = tf.Graph()
        with self.graph.as_default():
            if random_seed is not None:
                tf.set_random_seed(random_seed) # 设置随机种子
            self.X = tf.sparse_placeholder(dtype) 
            self.y = tf.placeholder(dtype)
            self.vars = utils.init_var_map(init_vars, init_path) # 初始化参数

            w = self.vars['w'] # ToDo [d*1]
            b = self.vars['b']# ToDo  [1]
            xw = tf.sparse_tensor_dense_matmul(self.X, w)# ToDo 稀疏向量的乘法 [n*d] * [d*1] = n*1
            logits = tf.reshape(xw + b, [-1])# ToDo shape=(n,1)
            self.y_prob = tf.sigmoid(logits)# ToDo

            self.loss = tf.reduce_mean(
                        tf.nn.sigmoid_cross_entropy_with_logits(labels=self.y, logits=logits)) + \
                        l2_weight * tf.nn.l2_loss(xw) # ToDo
            self.optimizer = utils.get_optimizer(opt_algo, learning_rate, self.loss)

            config = tf.ConfigProto()
            config.gpu_options.allow_growth = True
            self.sess = tf.Session(config=config)
            tf.global_variables_initializer().run(session=self.sess)

In [9]:
lr_params = {
            'input_dim': input_dim, # One-hot之后的所有维度之和
            'opt_algo': 'gd',
            'learning_rate': 0.001,
            'l2_weight': 0.1,
            'random_seed': 0
        }

model = LR(**lr_params)
train(model)

[0]	loss (with l2 norm):nan	train-auc: 0.482857	eval-auc: 0.293417
[1]	loss (with l2 norm):nan	train-auc: 0.479029	eval-auc: 0.263434
[2]	loss (with l2 norm):nan	train-auc: 0.480862	eval-auc: 0.242916
[3]	loss (with l2 norm):nan	train-auc: 0.484974	eval-auc: 0.228139
[4]	loss (with l2 norm):nan	train-auc: 0.492955	eval-auc: 0.217758
[5]	loss (with l2 norm):nan	train-auc: 0.498738	eval-auc: 0.211224
[6]	loss (with l2 norm):nan	train-auc: 0.504764	eval-auc: 0.205117
[7]	loss (with l2 norm):nan	train-auc: 0.512420	eval-auc: 0.200660
[8]	loss (with l2 norm):nan	train-auc: 0.517632	eval-auc: 0.196141
[9]	loss (with l2 norm):nan	train-auc: 0.523821	eval-auc: 0.194126
[10]	loss (with l2 norm):nan	train-auc: 0.530051	eval-auc: 0.191011
[11]	loss (with l2 norm):nan	train-auc: 0.535548	eval-auc: 0.190156
[12]	loss (with l2 norm):nan	train-auc: 0.541738	eval-auc: 0.188935
[13]	loss (with l2 norm):nan	train-auc: 0.546950	eval-auc: 0.188691
[14]	loss (with l2 norm):nan	train-auc: 0.553058	eval-auc:

#### 2. 请你补全下面 FM 类中缺失的代码，然后运行程序，输出AUC结果（30分）

In [12]:
class FM(Model):
    def __init__(self, input_dim=None, output_dim=1, factor_order=10, init_path=None, opt_algo='gd', learning_rate=1e-2,
                 l2_w=0, l2_v=0, random_seed=None):
        Model.__init__(self)
        init_vars = [('w', [input_dim, output_dim], 'tnormal', dtype),
                     ('v', [input_dim, factor_order], 'tnormal', dtype),
                     ('b', [output_dim], 'zero', dtype)]
        self.graph = tf.Graph()
        with self.graph.as_default():
            if random_seed is not None:
                tf.set_random_seed(random_seed)
            self.X = tf.sparse_placeholder(dtype)
            self.y = tf.placeholder(dtype)
            self.vars = utils.init_var_map(init_vars, init_path)

            w = self.vars['w'] # [input_dim, output_dim]
            v = self.vars['v'] # [input_dim, factor_order]
            b = self.vars['b'] # [output_dim]

            X_square = tf.SparseTensor(self.X.indices, tf.square(self.X.values), tf.to_int64(tf.shape(self.X))) # [batch_size, input_dim] 
            xv = tf.square(tf.sparse_tensor_dense_matmul(self.X, v)) # [n, input_dim] * [input_dim, factor_order] = n*factor_order
            p = 0.5 * tf.reshape(
                tf.reduce_sum(xv - tf.sparse_tensor_dense_matmul(X_square, tf.square(v)), 1), # 按行来求和
                [-1, output_dim])
            xw = tf.sparse_tensor_dense_matmul(self.X, w)
            logits = tf.reshape(xw + b + p, [-1])
            self.y_prob = tf.sigmoid(logits)

            self.loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=self.y)) + \
                        l2_w * tf.nn.l2_loss(xw) + \
                        l2_v * tf.nn.l2_loss(xv)
            self.optimizer = utils.get_optimizer(opt_algo, learning_rate, self.loss)

            config = tf.ConfigProto()
            config.gpu_options.allow_growth = True
            self.sess = tf.Session(config=config)
            tf.global_variables_initializer().run(session=self.sess)
            summary_writer = tf.summary.FileWriter("log", self.sess.graph)

In [13]:
fm_params = {
            'input_dim': input_dim,
            'factor_order': 10,
            'opt_algo': 'gd',
            'learning_rate': 0.1,
            'l2_w': 0,
            'l2_v': 0,
        }

model = FM(**fm_params)
train(model)

[0]	loss (with l2 norm):nan	train-auc: 0.585145	eval-auc: 0.711651
[1]	loss (with l2 norm):nan	train-auc: 0.589136	eval-auc: 0.715681
[2]	loss (with l2 norm):nan	train-auc: 0.591905	eval-auc: 0.718735
[3]	loss (with l2 norm):nan	train-auc: 0.593697	eval-auc: 0.721788
[4]	loss (with l2 norm):nan	train-auc: 0.594511	eval-auc: 0.723498
[5]	loss (with l2 norm):nan	train-auc: 0.595570	eval-auc: 0.725208
[6]	loss (with l2 norm):nan	train-auc: 0.597361	eval-auc: 0.726429
[7]	loss (with l2 norm):nan	train-auc: 0.598664	eval-auc: 0.726429
[8]	loss (with l2 norm):nan	train-auc: 0.599723	eval-auc: 0.726551
[9]	loss (with l2 norm):nan	train-auc: 0.601189	eval-auc: 0.726429
[10]	loss (with l2 norm):nan	train-auc: 0.602492	eval-auc: 0.726063
early stop
best iteration:
[8]	eval-auc: 0.726551


In [14]:
input_dim

560869

In [6]:
b = np.reshape(np.array([1,2,3,4,5,6]),[-1])
b

array([1, 2, 3, 4, 5, 6])